In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
# %matplotlib inline

#add
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from gensim.models import FastText, Word2Vec
import re
from keras.layers import *
from keras.models import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU, PReLU
import keras.backend as K
from keras.optimizers import *
from keras.utils import to_categorical

import keras.backend.tensorflow_backend as KTF
from scipy.sparse import csr_matrix, hstack, coo_matrix
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.


In [2]:
%%time
import gc
age_test = pd.read_csv('../age_test.csv', header = None, names = ['uId'])
age_train = pd.read_csv('../age_train.csv', header = None, names = ['uId','age_group'])
data = pd.concat([age_train,age_test], axis = 0,sort=True).reset_index()
data.drop(['index'],axis=1,inplace=True)
del age_test, age_train
gc.collect() 

CPU times: user 541 ms, sys: 159 ms, total: 699 ms
Wall time: 699 ms


In [3]:
user_app_actived = pd.read_csv('../user_app_actived.csv', header = None, names =['uId','appId'])

In [4]:
user_app_actived['app_list'] = user_app_actived['appId'].apply(lambda x: x.split('#'))

In [5]:
embed_size = 128
fastmodel = Word2Vec(list(user_app_actived['app_list']), size=embed_size, window=4, min_count=1, negative=2,
                     sg=1, sample=0.002, hs=1, workers=4)

In [6]:
embedding_fast = pd.DataFrame([fastmodel[word]for word in (fastmodel.wv.vocab)])
embedding_fast['app'] = list(fastmodel.wv.vocab)
embedding_fast.columns = ["fdim_%s" %str(i) for i in range(embed_size)]+["app"]
embedding_fast.shape

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(9401, 129)

In [7]:
data = data.merge(user_app_actived, on='uId', how='left')

In [8]:
test_index = np.isnan(data.age_group)
train_index = ~test_index
train_x = data[train_index] 
train_y = data[train_index]['age_group']
test_x  = data[test_index]

In [9]:
tokenizer = Tokenizer(lower=False, char_level=False, split='#')
tokenizer.fit_on_texts(list(user_app_actived['appId']))

In [10]:
X_seq = tokenizer.texts_to_sequences(train_x['appId'])
X_test_seq = tokenizer.texts_to_sequences(test_x['appId'])

In [11]:
maxlen = 50
X = pad_sequences(X_seq, maxlen=maxlen, value=0)
X_test = pad_sequences(X_test_seq, maxlen=maxlen, value=0)

In [27]:
max_feaures = 9402
embedding_matrix = np.zeros((max_feaures, embed_size))
for word in tokenizer.word_index:
    if word not in fastmodel.wv.vocab:
        continue
    embedding_matrix[tokenizer.word_index[word]] = fastmodel[word]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [46]:
class AdamW(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/4)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            # decoupled weight decay (2/4)
            self.wd = K.variable(weight_decay, name='weight_decay')
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd  # decoupled weight decay (3/4)

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            # decoupled weight decay (4/4)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - lr * wd * p

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def model_age_conv(embedding_matrix):

    # The embedding layer containing the word vectors
    K.clear_session()
    emb_layer = Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=maxlen,
        trainable=False
    )
    lstm_layer = Bidirectional(
        GRU(128, recurrent_dropout=0.15, dropout=0.15, return_sequences=True))

    # 1D convolutions that can iterate over the word vectors
    conv1 = Conv1D(filters=128, kernel_size=1,
                   padding='same', activation='relu',)
    conv2 = Conv1D(filters=64, kernel_size=2,
                   padding='same', activation='relu', )
    conv3 = Conv1D(filters=64, kernel_size=3,
                   padding='same', activation='relu',)
    conv5 = Conv1D(filters=32, kernel_size=5,
                   padding='same', activation='relu',)

    # Define inputs
    seq = Input(shape=(maxlen,))

    # Run inputs through embedding
    emb = emb_layer(seq)

    lstm = lstm_layer(emb)
    # Run through CONV + GAP layers
    conv1a = conv1(lstm)
    gap1a = GlobalAveragePooling1D()(conv1a)
    gmp1a = GlobalMaxPool1D()(conv1a)

    conv2a = conv2(lstm)
    gap2a = GlobalAveragePooling1D()(conv2a)
    gmp2a = GlobalMaxPool1D()(conv2a)

    conv3a = conv3(lstm)
    gap3a = GlobalAveragePooling1D()(conv3a)
    gmp3a = GlobalMaxPooling1D()(conv3a)

    conv5a = conv5(lstm)
    gap5a = GlobalAveragePooling1D()(conv5a)
    gmp5a = GlobalMaxPooling1D()(conv5a)

    merge1 = concatenate([gap1a, gap2a, gap3a, gap5a])

    # The MLP that determines the outcome
    x = Dropout(0.3)(merge1)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    pred = Dense(6, activation='softmax')(x)

    model = Model(inputs=seq, outputs=pred)
    model.compile(loss='categorical_crossentropy',optimizer=AdamW(weight_decay=0.08,),metrics=['accuracy'])

    return model

In [29]:
kfold = StratifiedKFold(n_splits=5, random_state=20, shuffle=True)

In [30]:
from sklearn.preprocessing import LabelEncoder
train_x['age_group'] = LabelEncoder().fit_transform(train_x['age_group'])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
def batch_generator(X, y, batch_size, shuffle):
    #chenglong code for fiting from generator (https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22567/neural-network-for-sparse-matrices)
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:]
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch
        if (counter == number_of_batches):
            counter = 0

In [49]:
Y_age = to_categorical(train_x['age_group'])

sub2 = np.zeros((X_test.shape[0], 6))
oof_pref2 = np.zeros((X.shape[0], 6))
score = []
count = 0
for i, (train_index, test_index) in enumerate(kfold.split(X, train_x['age_group'])):

    print("FOLD | ", count+1)

    filepath2 = "age_weights_best_%d.h5" % count
    checkpoint2 =  ModelCheckpoint(filepath='./weight_bias/best_epoch_model_actived_app.hdf5',monitor='val_acc',mode='max' ,save_best_only=True,verbose=1,period=1)
    reduce_lr2 = ReduceLROnPlateau(monitor='val_acc', factor=0.8, patience=2, min_lr=0.0001, verbose=1)
    earlystopping2 = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=8, verbose=1, mode='auto')
    callbacks2 = [checkpoint2, reduce_lr2, earlystopping2]
    checkpoint = ModelCheckpoint(filepath='./weight_bias/best_epoch_model_actived_app.hdf5',monitor='val_acc',mode='max' ,save_best_only=True,verbose=1,period=1)
     
    callback_lists=[checkpoint]
    X_tr, X_vl, y_tr, y_vl = X[train_index], X[test_index], Y_age[train_index], Y_age[test_index]

    model_age = model_age_conv(embedding_matrix)
    hist = model_age.fit_generator(generator=batch_generator(X_tr, y_tr, 512, True),
                         samples_per_epoch=np.ceil(X_tr.shape[0]/512),
                         epochs=50, validation_data=(X_vl, y_vl),
                         callbacks=callbacks2, verbose=1, shuffle=True)

    model_age.load_weights(filepath2)
    oof_pref2[test_index] = model_age.predict(X_vl)
    sub2 += model_age.predict(X_test)/kfold.n_splits
    score.append(np.min(hist.history['val_loss']))
    count += 1
print('log loss:', np.mean(score))

FOLD |  1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., epochs=50, validation_data=(array([[ ..., callbacks=[<keras.ca..., verbose=1, shuffle=True, steps_per_epoch=3141.0)`


Epoch 1/50
3141/3141 [==============================] - 580s 185ms/step - loss: 1.0750 - acc: 0.5448 - val_loss: 0.9850 - val_acc: 0.5836

Epoch 00001: val_acc improved from -inf to 0.58358, saving model to ./weight_bias/best_epoch_model_actived_app.hdf5
Epoch 2/50
3141/3141 [==============================] - 571s 182ms/step - loss: 0.9994 - acc: 0.5774 - val_loss: 0.9933 - val_acc: 0.5806

Epoch 00002: val_acc did not improve from 0.58358
Epoch 3/50
3141/3141 [==============================] - 577s 184ms/step - loss: 0.9842 - acc: 0.5842 - val_loss: 0.9684 - val_acc: 0.5902

Epoch 00003: val_acc improved from 0.58358 to 0.59024, saving model to ./weight_bias/best_epoch_model_actived_app.hdf5
Epoch 4/50
3141/3141 [==============================] - 569s 181ms/step - loss: 0.9768 - acc: 0.5879 - val_loss: 0.9542 - val_acc: 0.5973

Epoch 00004: val_acc improved from 0.59024 to 0.59735, saving model to ./weight_bias/best_epoch_model_actived_app.hdf5
Epoch 5/50
3141/3141 [==================

KeyboardInterrupt: 